In [1]:
!pip3 install mcerp
import numpy as np
import scipy
import mcerp
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'numpy'

In [ ]:
NORMALIZATION_SERVER = 200
NORMALIZATION_COOLING = 20000
#ROOM_TEMPERATURE = 24

min_load = 0
max_load = 100
expected_mean = 60

# https://en.wikipedia.org/wiki/PERT_distribution
# <b> is the load min
b = (6 * expected_mean - min_load - max_load) / 4
xv = mcerp.PERT(min_load, b, max_load)
xv.plot()

In [ ]:
from mcerp.lhd import lhd

random_values = lhd(dist=xv.rv, size=48)
#random_values = [34, 14, 46, 9, 23, 6, 12, 17, 8, 30, 38, 16, 19, 8, 50, 27, 18, 11, 6, 11, 21, 41, 5, 14, 23, 15, 25, 31, 28, 35, 1, 55, 9, 3, 25, 19, 32, 2, 13, 5, 3, 39, 12, 61, 7, 21, 4, 1]
print(random_values.tolist())
print("mean with raw values: %s" % np.mean(random_values))
rounded_random_values = [np.around(x) for x in random_values]
print("mean with rounded values: %s" % np.mean(rounded_random_values))
random_values = rounded_random_values

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# sphinx_gallery_thumbnail_number = 2

rows = ["row %s" % x for x in range(1,13)]
racks = ["Z1.5", "Z1.4", "Z1.2", "Z1.1"]

def compute_low_top_placement(loads):
    sorted_loads = sorted(loads)
    result = np.array(sorted_loads).reshape(12,4)
    return result

def compute_low_down_placement(loads):
    sorted_loads = sorted(loads, reverse=True)
    result = np.array(sorted_loads).reshape(12,4)
    return result

def compute_high_center_placement(loads):
    sorted_loads = [x.item() for x in sorted(loads)]
    placement_idx = [
        [0, 1, 2, 3], #1
        [4, 28, 29, 5], #2
        [6, 32, 33, 7], #3
        [8, 36, 37, 9], #4
        [10, 40, 41, 11], #5
        [12, 44, 45, 13], #6
        [14, 46, 47, 15], #7
        [16, 42, 43, 17], #8
        [18, 38, 39, 19], #9
        [20, 34, 35, 21], #10
        [22, 30, 31, 23], #11
        [24, 25, 26, 27], #12
    ]
    
    placement = [ [sorted_loads[idx] for idx in row] for row in placement_idx ] 
    result = np.array(placement)
    return result

placement = compute_low_top_placement(random_values)
#placement = compute_low_down_placement(random_values)
#placement = compute_high_center_placement(random_values)


fig, ax = plt.subplots()
im = ax.imshow(placement)

# We want to show all ticks...
ax.set_xticks(np.arange(len(racks)))
ax.set_yticks(np.arange(len(rows)))
# ... and label them with the respective list entries
ax.set_xticklabels(racks)
ax.set_yticklabels(rows)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(rows)):
    for j in range(len(racks)):
        text = ax.text(j, i, int(placement[i, j]),
                       ha="center", va="center", color="w")

ax.set_title("Placement of loads on Racks")
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots()

flat_placement = np.reshape(placement, (1, 48), order='F')
print(flat_placement)
im = ax.imshow(flat_placement)

plt.show()

In [ ]:
import math

def load_to_power_consumption(load):
    #power_consumption = 99.2314747103 + 2.05275371*load - 0.01289649*load*load
    power_consumption = 1.45331636e+02 + load * 2.96239188e-01 - 7.69497994e+01 * np.exp(-1 * 7.99954988e-02 * load)
    return power_consumption
    
power_consumptions = [load_to_power_consumption(x) for x in placement]
fig, ax = plt.subplots()

flat_power_consumptions = np.reshape(power_consumptions, (1, 48), order='F')
print(flat_power_consumptions)
print(np.mean(flat_power_consumptions))
im = ax.imshow(flat_placement)

plt.show()

In [ ]:
x = np.linspace(0, 100, 500)
y = load_to_power_consumption(x)

fig, ax = plt.subplots()

# Using set_dashes() to modify dashing of an existing line
line1, = ax.plot(x, y, label='consumption')

ax.legend()
plt.show()

In [ ]:
normalized_flat_power_consumption = [x / NORMALIZATION_SERVER for x in flat_power_consumptions]
print(normalized_flat_power_consumption)

In [ ]:
from keras.models import load_model

# returns a compiled model
# identical to the previous one
#oracle = load_model('/Users/jonathan/seduceml3.h5')
#oracle = load_model('/Users/jonathan/Documents/workspace/seduce_ml/data/seduceml_2019_06_17_T_11_28_30.h5')
#oracle = load_model('/Users/jonathan/Documents/workspace/seduce_ml/data/seduceml_2019_06_20_T_00_04_32.h5')

#oracle = load_model('/Users/jonathan/Documents/workspace/seduce_ml/data/seduceml_2019_06_26_T_15_03_29.h5')
#oracle = load_model('/Users/jonathan/Documents/workspace/seduce_ml/data/seduceml_2019_06_27_T_15_53_30.h5')

#oracle = load_model('/Users/jonathan/Documents/workspace/seduce_ml/data/seduceml_2019_07_02_T_10_57_43.h5')
#oracle = load_model('/Users/jonathan/Documents/workspace/seduce_ml/data/seduceml_2019_07_02_T_13_48_13.h5')
#oracle = load_model('/Users/jonathan/Documents/workspace/seduce_ml/data/seduceml_2019_07_04_T_10_05_15.h5')

#oracle = load_model('/Users/jonathan/Documents/workspace/seduce_ml/data/seduceml_2019_07_05_T_19_36_30.h5')
oracle = load_model('../data/seduceml_2019_07_14_T_12_07_51.h5')

In [ ]:
def prepare_load_array(load_array, temperature):
    return np.array([load_array[0]])

def compute_oracle_based_placement(loads, oracle):
    free_positions = range(0, 48)
    sorted_loads = [x.item() for x in sorted(loads, reverse=True)]
    
    #return sorted_loads
    
    mean_load = np.mean(sorted_loads)
    adjusted_mean_load = mean_load * 48 / 47

    #result = [0.0 for x in range(0, 48)]
    
    result = [adjusted_mean_load for x in range(0, 48)]
    for load in sorted_loads:
        current_best_score = 100
        current_best_free_position = -1
        for free_position in free_positions:
            temp_result = result[:]
            temp_result[free_position] = load
            
            # Do computation
            power_consumptions = [load_to_power_consumption(x) for x in temp_result]
            normalized_power_consumptions = [x / NORMALIZATION_SERVER for x in power_consumptions]
            
            #current_score = oracle.predict([normalized_flat_power_consumption])[0][0]
            proposed_loads = [np.array(normalized_power_consumptions)]
            proposed_loads_with_temperature = add_temperature_to_load_array(proposed_loads, ROOM_TEMPERATURE)
            
            current_score = np.max(oracle.predict(proposed_loads_with_temperature))
            
            if current_best_free_position == -1 or current_score < current_best_score:
                current_best_score = current_score
                current_best_free_position = free_position
        
        # Use the best result
        free_positions = [x for x in free_positions if x != current_best_free_position]
        result[current_best_free_position] = load
    
    
    print(current_best_score)
    placement = np.reshape(result, (12,4), order="F")
    
    return placement

normalized_flat_power_consumption_with_temp = prepare_load_array(normalized_flat_power_consumption, ROOM_TEMPERATURE)
print("mean: %s" % np.mean(oracle.predict(normalized_flat_power_consumption_with_temp)))
print("std: %s" % np.std(oracle.predict(normalized_flat_power_consumption_with_temp)))
print("vs")
oracle_based_placement = np.array(compute_oracle_based_placement(random_values, oracle)).reshape(12, 4)

oracle_power_consumptions = [load_to_power_consumption(x) for x in oracle_based_placement]
oracle_power_consumptions = np.reshape(oracle_power_consumptions, (1, 48), order='F')

normalized_oracle_power_consumptions = [x / NORMALIZATION_SERVER for x in oracle_power_consumptions]
normalized_oracle_power_consumptions = prepare_load_array(normalized_oracle_power_consumptions, 12)

print(normalized_flat_power_consumption_with_temp)
print(normalized_oracle_power_consumptions)

print("mean: %s" % np.mean(oracle.predict(normalized_oracle_power_consumptions)))
print("std: %s" % np.std(oracle.predict(normalized_oracle_power_consumptions)))

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(oracle_based_placement)

# We want to show all ticks...
ax.set_xticks(np.arange(len(racks)))
ax.set_yticks(np.arange(len(rows)))
# ... and label them with the respective list entries
ax.set_xticklabels(racks)
ax.set_yticklabels(rows)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(rows)):
    for j in range(len(racks)):
        text = ax.text(j, i, int(oracle_based_placement[i, j]),
                       ha="center", va="center", color="w")

ax.set_title("Placement of loads on Racks")
fig.tight_layout()
plt.show()

In [ ]:
print(oracle_based_placement)
optimised_placement = oracle_based_placement.reshape((1, 48), order="F")
print(optimised_placement)

In [ ]:
print([int(x) for x in flat_placement[0]])
print([int(x) for x in optimised_placement[0]])